<a href="https://colab.research.google.com/github/Mohamedh0/Amit/blob/main/Fake_News_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77


In [ ]:
# Important Libraries
import string
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding,Dense,LSTM, Dropout,Bidirectional
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#Load the data
df=pd.read_csv(r'/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv',index_col=0)
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72134 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71576 non-null  object
 1   text    72095 non-null  object
 2   label   72134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
# Drop Null values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71537 entries, 0 to 72133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   71537 non-null  object
 1   text    71537 non-null  object
 2   label   71537 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
# Split the data
X=df['text']
y=df['label']

In [ ]:
def normalize_text(text):
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # 3. Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # 4. Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 5. Tokenize the text
    tokens = word_tokenize(text)

    # 6. Remove stopwords and numeric tokens
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [
        token for token in tokens
        if token not in stop_words and token.isalpha()
    ]

    # 7. Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    return ' '.join(lemmatized_tokens)

In [ ]:
# Preprocessing Text
X=X.apply(normalize_text)
X.head()

,text
0,comment expected barack obama member fyf fukyo...
2,demonstrator gathered last night exercising co...
3,dozen politically active pastor came private d...
4,r sarmat missile dubbed satan replace s fly mi...
5,say one time someone sued southern poverty law...


In [ ]:
# Making Training and Testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(X_train)
# Calculate vocabulary size
vocab_size = len(t.word_index) + 1
max_len=100
# Integer encode
encoded_train_docs = t.texts_to_sequences(X_train)
# Integer encode
encoded_test_docs = t.texts_to_sequences(X_test)

# Pad documents to a max length of 100 words
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_len, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_len, padding='post')

# Load the whole embedding into memory
embeddings_index = dict()
with open('glove.6B.100d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print(f'Loaded {len(embeddings_index)} word vectors.')

# Create a weight matrix for words
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



Loaded 400000 word vectors.


In [ ]:
# Define the enhanced model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model to see the number of parameters
model.summary()
model.fit(padded_train_docs, y_train, epochs=5, verbose=1)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │      27,037,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 27,037,500 (103.14 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 27,037,500 (103.14 MB)

Epoch 1/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 1013s 564ms/step - accuracy: 0.7484 - loss: 0.5027
Epoch 2/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 1031s 558ms/step - accuracy: 0.8775 - loss: 0.3113
Epoch 3/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 1060s 567ms/step - accuracy: 0.9239 - loss: 0.1971
Epoch 4/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 1081s 589ms/step - accuracy: 0.9313 - loss: 0.1810
Epoch 5/5
1789/1789 ━━━━━━━━━━━━━━━━━━━━ 1044s 557ms/step - accuracy: 0.9342 - loss: 0.1766


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print(f'Test Accuracy: {(accuracy)}, Loss: {loss}')

Test Accuracy: 0.9461140632629395, Loss: 0.13763922452926636
